# Projekt "Credit Risk Modeling"

## 1. Daten erkunden

In [1]:
import pandas as pd
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv("german_credit_data.csv")
df

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...,...
995,995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good
996,996,40,male,3,own,little,little,3857,30,car,good
997,997,38,male,2,own,little,NaN,804,12,radio/TV,good
998,998,23,male,2,free,little,little,1845,45,radio/TV,bad


In [3]:
df.describe()

,Unnamed: 0,Age,Job,Credit amount,Duration
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,35.546000,1.904000,3271.258000,20.903000
std,288.819436,11.375469,0.653614,2822.736876,12.058814
min,0.000000,19.000000,0.000000,250.000000,4.000000
25%,249.750000,27.000000,2.000000,1365.500000,12.000000
50%,499.500000,33.000000,2.000000,2319.500000,18.000000
75%,749.250000,42.000000,2.000000,3972.250000,24.000000
max,999.000000,75.000000,3.000000,18424.000000,72.000000


(Unnamed 0), hat keinen inhaltlichen Mehrwert und wird später entfernt.

In [4]:
print(df["Risk"].value_counts())

Risk
good    700
bad     300
Name: count, dtype: int64


Daten sind unausgeglichen muss zum Trainieren des Modells beachtet werden.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1000 non-null   int64 
 1   Age               1000 non-null   int64 
 2   Sex               1000 non-null   object
 3   Job               1000 non-null   int64 
 4   Housing           1000 non-null   object
 5   Saving accounts   817 non-null    object
 6   Checking account  606 non-null    object
 7   Credit amount     1000 non-null   int64 
 8   Duration          1000 non-null   int64 
 9   Purpose           1000 non-null   object
 10  Risk              1000 non-null   object
dtypes: int64(5), object(6)
memory usage: 86.1+ KB


Nachdem wir uns einen Überblick über den Datensatz verschafft haben, wollen wir nun genauer in eine einzelne Spalte schauen

In [6]:
df["Job"].unique()

array([2, 1, 3, 0])

In [7]:
df.isna().sum()

Unnamed: 0            0
Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64

In [8]:
df.duplicated().sum()

np.int64(0)

Jetzt entfernen ich alle Zeilen, die mindestens einen fehlenden Wert enthalten. Dadurch bleiben nur noch vollständig ausgefüllte Datensätze übrig

In [9]:
df = df.dropna().reset_index(drop=True)

In [10]:
df.isna().sum()

Unnamed: 0          0
Age                 0
Sex                 0
Job                 0
Housing             0
Saving accounts     0
Checking account    0
Credit amount       0
Duration            0
Purpose             0
Risk                0
dtype: int64

In [11]:
df.drop(columns="Unnamed: 0", inplace=True)

In [12]:
df.columns

Index(['Age', 'Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account',
       'Credit amount', 'Duration', 'Purpose', 'Risk'],
      dtype='object')

## Verteilung der numerischen Merkmale

In [13]:
# 1) Numerische Spalten ermitteln:
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()

# 2) Eigene Bins je Merkmal:
bins_cfg = {
    "Age":            dict(start=15, end=80,    size=5),  
    "Credit amount":  dict(start=0,  end=20000, size=1000), 
    "Duration":       dict(start=0,  end=80,    size=6),
    "Job":            dict(start=-0.5, end=3.5, size=1),
}

# 3) Subplot-Grid abhängig von der Anzahl der numerischen Spalten:
cols_per_row = 2
rows = math.ceil(len(numeric_cols) / cols_per_row)

fig1 = make_subplots(
    rows=rows,
    cols=cols_per_row,
    subplot_titles=numeric_cols,
    horizontal_spacing=0.08,
    vertical_spacing=0.15
)

# 4) Histogramme hinzufügen:
for i, col in enumerate(numeric_cols):
    r = i // cols_per_row + 1
    c = i % cols_per_row + 1
    fig1.add_trace(
        go.Histogram(
            x=df[col],
            xbins=bins_cfg.get(col, None),
            marker_line_color="black",
            marker_line_width=1,
            name=col
        ),
        row=r, col=c
    )

# 5) Optik anpassen:
fig1.update_xaxes(showgrid=False)
fig1.update_yaxes(showgrid=False)
fig1.update_layout(
    title="Verteilung der numerischen Merkmale",
    showlegend=False,
    bargap=0.05,
    height=350 * rows 
)

fig1

- Age: Die Verteilung ist rechtsschief: Viele jüngere Menschen, immer weniger mit zunehmendem Alter.
- Job: Nicht gleichmäßige Verteilung: Kategorie 2 dominiert stark, während 0 und 3 selten sind. Der Schwerpunkt liegt deutlich auf mittleren Jobs.
- Credit amount: Sehr stark rechtsschief: Die meisten Kredite sind klein (unter 5.000 €), nur wenige sehr groß. Die Ausreißer nach oben ziehen den Wertebereich stark auseinander.
- Duration: Ebenfalls rechtsschief: Kürzere Laufzeiten (bis 24 Monate) dominieren. Lange Laufzeiten sind selten und bilden Ausreißer.

In [14]:
# 1) Subplot-Raster festlegen:
cols_per_row = 2
rows = math.ceil(len(numeric_cols) / cols_per_row)

fig2 = make_subplots(
    rows=rows,
    cols=cols_per_row,
    subplot_titles=numeric_cols,
    horizontal_spacing=0.08,
    vertical_spacing=0.15
)

# 2) Boxplots je numerischer Spalte hinzufügen:
for i, col in enumerate(numeric_cols):
    r = i // cols_per_row + 1
    c = i % cols_per_row + 1
    fig2.add_trace(
        go.Box(
            y=df[col],
            name=col,
            boxpoints="outliers",   
            marker_line_color="black",
            marker_line_width=1,
            whiskerwidth=0.8
        ),
        row=r, col=c
    )

# 3) Layout anpassen:
fig2.update_xaxes(showgrid=False)
fig2.update_yaxes(showgrid=True)
fig2.update_layout(
    title="Boxplots der numerischen Merkmale",
    showlegend=False,
    height=300 * rows
)

fig2

Die Boxplots zeigen, dass es einige Ausreißer bei der Kreditlaufzeit (Duration) gibt, die über 70 Monate liegen.

In [15]:
df.query("Duration > 70")

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
358,24,male,2,own,moderate,moderate,5595,72,radio/TV,bad


In [16]:
df.query("Duration >= 60")

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
18,63,male,2,own,little,little,6836,60,business,bad
176,24,female,3,own,moderate,moderate,7408,60,car,bad
199,60,female,3,free,moderate,moderate,14782,60,vacation/others,bad
358,24,male,2,own,moderate,moderate,5595,72,radio/TV,bad
378,27,male,3,own,little,moderate,14027,60,car,bad
489,42,male,2,free,little,moderate,6288,60,education,bad
507,36,male,2,rent,little,little,7297,60,business,bad


## Kategorialen Merkmale untersuchen

In [17]:
# 1) Kategorische Spalten ermitteln:
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()

# 2) Subplot-Raster anlegen:
cols_per_row = 3
rows = math.ceil(len(categorical_cols) / cols_per_row)

fig3 = make_subplots(
    rows=rows,
    cols=cols_per_row,
    subplot_titles=categorical_cols,
    horizontal_spacing=0.08,
    vertical_spacing=0.15
)

# 3) Für jede kategoriale Spalte ein Balkendiagramm hinzufügen:
for i, col in enumerate(categorical_cols):
    r = i // cols_per_row + 1
    c = i % cols_per_row + 1
    
    value_counts = df[col].value_counts().sort_values(ascending=False)
    
    fig3.add_trace(
        go.Bar(
            x=value_counts.index.astype(str), 
            y=value_counts.values,             
            marker_line_color="black",
            marker_line_width=1,
            name=col
        ),
        row=r, col=c
    )

# 4) Layout anpassen:
fig3.update_xaxes(showgrid=False, tickangle=45)
fig3.update_yaxes(showgrid=False)
fig3.update_layout(
    title="Verteilung der kategorialen Merkmale",
    showlegend=False,
    height=350 * rows
)

fig3

Korrelationsmatrix

In [18]:
corr = df[numeric_cols].corr()
corr

,Age,Job,Credit amount,Duration
Age,1.000000,0.039771,0.082014,0.001549
Job,0.039771,1.000000,0.334721,0.200794
Credit amount,0.082014,0.334721,1.000000,0.613298
Duration,0.001549,0.200794,0.613298,1.000000


In [19]:
corr = df[numeric_cols].corr().round(2)

fig4 = px.imshow(
    corr,
    text_auto=True,       
    color_continuous_scale="RdBu_r",
    zmin=-1, zmax=1,   
    aspect="auto",
    title="Korrelationsmatrix der numerischen Merkmale"
)

fig4.update_layout(margin=dict(l=60, r=20, t=60, b=40))
fig4

Gibt es einen Zusammenhang zwischen der Job-Kategorie und der Höhe des aufgenommenen Kredits

In [20]:
df.groupby("Job")["Credit amount"].mean()

Job
0    1767.857143
1    2250.715517
2    3129.130990
3    5648.784810
Name: Credit amount, dtype: float64

Die Analyse zeigt einen klaren Zusammenhang zwischen Job-Kategorie und Kredithöhe. Menschen in höheren Job-Kategorien nehmen im Durchschnitt deutlich größere Kredite auf als Personen in niedrigeren Kategorien.



In [21]:
df.groupby("Sex")["Credit amount"].mean()

Sex
female    2937.202381
male      3440.833333
Name: Credit amount, dtype: float64

Männer nehmen in diesem Datensatz im Durchschnitt höhere Kredite auf als Frauen. Der Unterschied ist zwar sichtbar, aber nicht so groß, dass er allein entscheidend wäre.

In [22]:
pd.pivot_table(df, values="Credit amount", index="Housing", columns="Purpose")

Purpose,business,car,domestic appliances,education,furniture/equipment,radio/TV,repairs,vacation/others
Housing,,,,,,,,
free,4705.000000,5180.314286,NaN,5314.250000,4419.444444,2097.000000,1190.0,7842.666667
own,3725.973684,3120.485437,1333.5,2625.076923,3031.100000,2307.613861,2993.5,10321.833333
rent,6180.833333,3398.285714,NaN,2627.857143,2890.285714,2138.000000,2384.0,NaN


In [23]:
fig5 = px.scatter(
    df,
    x="Age",
    y="Credit amount",
    color="Sex",       
    size="Duration",   
    opacity=0.7,         
    title="Alter vs. Kreditbetrag",
    labels={"Age": "Alter", "Credit amount": "Kreditbetrag"}
)

fig5.update_layout(
    legend_title="Geschlecht",
    margin=dict(l=40, r=40, t=60, b=40)
)

fig5

Die Mehrheit der Kredite bewegt sich im niedrigen bis mittleren Bereich, junge Kreditnehmer sind stärker vertreten, und hohe Kredite gehen meist mit langen Laufzeiten einher.


In [24]:
fig6 = px.violin(
    df,
    x="Saving accounts",
    y="Credit amount",
    box=True,      
    points="all",        
    color="Saving accounts",
    title="Verteilung der Kreditbeträge nach Sparguthaben",
    labels={"Saving accounts": "Sparguthaben", "Credit amount": "Kreditbetrag"}
)

fig6.update_layout(showlegend=False, margin=dict(l=40, r=40, t=60, b=40))
fig6

Bei den Kategorien moderate, quite rich und rich gibt es zwar insgesamt weniger Fälle, die Muster sind aber ähnlich: Die meisten Kredite liegen im unteren bis mittleren Bereich, und es tauchen immer wieder einzelne Ausreißer nach oben auf. Daraus können wir schließen, dass die Höhe der Ersparnisse nicht allein bestimmt, wie hoch ein Kredit ist. In allen Gruppen finden sich sowohl kleine als auch große Kreditbeträge, sodass andere Faktoren – zum Beispiel Beruf oder Einkommen – wahrscheinlich wichtiger sind.


In [25]:
print(df["Risk"].value_counts(normalize=True) * 100)

Risk
good    55.747126
bad     44.252874
Name: proportion, dtype: float64


Die Auswertung zeigt, dass etwa 56 % der Kredite als good (niedriges Risiko) und rund 44 % als bad (hohes Risiko) eingestuft sind. Damit ist der Datensatz zwar nicht perfekt ausgeglichen, aber beide Klassen sind ausreichend stark vertreten.

In [26]:
cols = ["Age", "Credit amount", "Duration"]

# Subplot-Raster (1 Reihe, 3 Spalten):
fig7 = make_subplots(
    rows=1, cols=3,
    subplot_titles=cols
)

# Für jede Spalte einen Boxplot hinzufügen:
for i, col in enumerate(cols):
    fig7.add_trace(
        go.Box(
            x=df["Risk"],
            y=df[col],
            boxpoints="outliers",   
            marker=dict(color="lightblue"),
            line=dict(color="black"),
            name=col
        ),
        row=1, col=i+1
    )

fig7.update_layout(
    title="Boxplots der Merkmale nach Risiko-Klasse",
    showlegend=False,
    margin=dict(l=40, r=40, t=60, b=40)
)

fig7

In den drei Boxplots sieht man den Vergleich der Merkmale Alter, Kreditbetrag und Laufzeit zwischen den Risiko-Klassen good (niedriges Risiko) und bad (hohes Risiko).

- Beim Alter erkennt man kaum Unterschiede: Sowohl bei „good“ als auch bei „bad“ liegen die meisten Personen zwischen 25 und 40 Jahren. Das Alter allein scheint also kein starker Einflussfaktor auf das Risiko zu sein.
- Beim Kreditbetrag zeigt sich, dass Kunden mit hohem Risiko (bad) im Durchschnitt höhere Kredite aufnehmen und auch deutlich mehr Ausreißer nach oben haben (teilweise über 15.000 €). Das spricht dafür, dass sehr hohe Kredite häufiger riskant sind.
- Besonders deutlich wird der Unterschied bei der Laufzeit (Duration): Kredite mit hohem Risiko haben im Mittel deutlich längere Laufzeiten. Während „good“-Kredite meist kürzer laufen, sieht man bei „bad“ viele Verträge mit 40 Monaten oder mehr.

Alter spielt für das Risiko nur eine geringe Rolle, während hohe Kreditbeträge und vor allem lange Laufzeiten stärker mit einem höheren Risiko verbunden sind. Nun wollen wir diese Unterschiede auch zahlenmäßig festhalten:

In [27]:
print(df.groupby("Risk")[["Age", "Credit amount", "Duration"]].mean())

            Age  Credit amount   Duration
Risk                                     
bad   34.147186    3881.090909  25.445887
good  35.477663    2800.594502  18.079038


Die Berechnung zeigt deutliche Unterschiede zwischen den Risiko-Klassen. Personen mit hohem Risiko (bad) haben im Durchschnitt etwas weniger Alter, dafür aber höhere Kreditbeträge (ca. 3.881 € gegenüber 2.801 €) und vor allem deutlich längere Laufzeiten (25 Monate gegenüber 18 Monaten). Das bestätigt, was wir zuvor in den Boxplots gesehen haben: lange Laufzeiten und hohe Kreditbeträge erhöhen das Risiko eines Kredits, während das Alter nur eine kleine Rolle spielt.

In [28]:
# Subplot-Raster (3 Spalten pro Reihe):
cols_per_row = 3
rows = math.ceil(len(categorical_cols) / cols_per_row)

fig8 = make_subplots(
    rows=rows,
    cols=cols_per_row,
    subplot_titles=categorical_cols,
    horizontal_spacing=0.08,
    vertical_spacing=0.15
)

for i, col in enumerate(categorical_cols):
    r = i // cols_per_row + 1
    c = i % cols_per_row + 1
    
    if col != "Risk":
        # Häufigkeiten nach Risk berechnen:
        value_counts = df.groupby([col, "Risk"]).size().reset_index(name="count")
        
        # Balken hinzufügen (good vs. bad):
        for risk_class in value_counts["Risk"].unique():
            subset = value_counts[value_counts["Risk"] == risk_class]
            fig8.add_trace(
                go.Bar(
                    x=subset[col].astype(str),
                    y=subset["count"],
                    name=risk_class,
                    marker_line_color="black",
                    marker_line_width=1
                ),
                row=r, col=c
            )
    else:
        # Nur einfache Verteilung von Risk selbst:
        value_counts = df[col].value_counts().reset_index()
        value_counts.columns = [col, "count"]
        
        fig8.add_trace(
            go.Bar(
                x=value_counts[col].astype(str),
                y=value_counts["count"],
                name=col,
                marker_line_color="black",
                marker_line_width=1
            ),
            row=r, col=c
        )

fig8.update_xaxes(showgrid=False, tickangle=45)
fig8.update_yaxes(showgrid=False)
fig8.update_layout(
    title="Verteilung der kategorialen Merkmale",
    barmode="group",
    height=350 * rows
)

fig8

## Modell vorbereitung

In [29]:
features= ["Age", "Sex", "Job", "Housing", "Saving accounts", "Checking account", "Credit amount", "Duration"]
target = "Risk"
df_model = df[features + [target]].copy()
df_model


,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Risk
0,22,female,2,own,little,moderate,5951,48,bad
1,45,male,2,free,little,little,7882,42,good
2,53,male,2,free,little,little,4870,24,bad
3,35,male,3,rent,little,moderate,6948,36,good
4,28,male,3,own,little,moderate,5234,30,bad
...,...,...,...,...,...,...,...,...,...
517,48,male,1,own,little,moderate,1743,24,good
518,30,male,3,own,little,little,3959,36,good
519,40,male,3,own,little,little,3857,30,good
520,23,male,2,free,little,little,1845,45,bad


In [30]:
cat_cols = df_model.select_dtypes(include=["object"]).columns.drop("Risk")
le_dict = {}
for col in cat_cols:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])
    le_dict[col] = le
    joblib.dump(le, f"{col}_encoder.pkl")

df_model

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Risk
0,22,0,2,1,0,1,5951,48,bad
1,45,1,2,0,0,0,7882,42,good
2,53,1,2,0,0,0,4870,24,bad
3,35,1,3,2,0,1,6948,36,good
4,28,1,3,1,0,1,5234,30,bad
...,...,...,...,...,...,...,...,...,...
517,48,1,1,1,0,1,1743,24,good
518,30,1,3,1,0,0,3959,36,good
519,40,1,3,1,0,0,3857,30,good
520,23,1,2,0,0,0,1845,45,bad


In [31]:
le_target = LabelEncoder()
df_model[target] = le_target.fit_transform(df_model[target])

df_model[target]

0      0
1      1
2      0
3      1
4      0
      ..
517    1
518    1
519    1
520    0
521    1
Name: Risk, Length: 522, dtype: int64

In [32]:
print(df_model[target].value_counts())

Risk
1    291
0    231
Name: count, dtype: int64


In [33]:
joblib.dump(le_target, "target_encoder.pkl")

['target_encoder.pkl']

In [34]:
X = df_model.drop(target, axis=1)
y = df_model[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f"Geometrie von 'X': {X.shape}")
print(f"Geometrie von 'y': {y.shape}")

Geometrie von 'X': (522, 8)
Geometrie von 'y': (522,)


In [35]:
# Hielfsfunktion
def train_model(model, param_grid, X_train, y_train, X_test, y_test):
    grid = GridSearchCV(model, param_grid, cv=5, scoring="accuracy", n_jobs=-1)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return best_model, acc, grid.best_params_

In [36]:
dt = DecisionTreeClassifier(random_state=42, class_weight="balanced")
dt_param_grid = {
    "max_depth": [3, 5, 7, 10, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

In [37]:
best_dt, acc_dt, params_dt = train_model(dt, dt_param_grid, X_train, y_train, X_test, y_test)

In [38]:
print(f"Decision Tree Accuaracy: {acc_dt}")
print(f"Best parameters:\n{params_dt}")

Decision Tree Accuaracy: 0.6
Best parameters:
{'max_depth': 7, 'min_samples_leaf': 2, 'min_samples_split': 10}


Das Modell liegt in etwa bei 60% richtiger Vorhersagen. Das ist ein Anfang, aber noch nicht besonders hoch.

In [39]:
rf = RandomForestClassifier(random_state=42, class_weight="balanced", n_jobs=-1)
rf_param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [5, 7, 10, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}
best_rf, acc_rf, params_rf = train_model(rf, rf_param_grid, X_train, y_train, X_test, y_test)
print(f"Random Forest Accuaracy: {acc_rf}")
print(f"Best parameters:\n{params_rf}")

Random Forest Accuaracy: 0.6285714285714286
Best parameters:
{'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}


Der Random Forest verbessert die Vorhersage im Vergleich zum einzelnen Entscheidungsbaum: Die Accuracy steigt auf ca. 0,63 (vorher ~0,60). Das ist typisch für Ensemble-Methoden: Viele Bäume mitteln individuelle Fehler und liefern stabilere Ergebnisse. Die besten Hyperparameter deuten auf ein eher konservatives Modell hin.

In [40]:
et = ExtraTreesClassifier(random_state=42, class_weight="balanced", n_jobs=-1)
et_param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [5, 7, 10, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}
best_et, acc_et, params_et = train_model(et, et_param_grid, X_train, y_train, X_test, y_test)
print(f"Extra trees Accuaracy: {acc_et}")
print(f"Best parameters:\n{params_et}")

Extra trees Accuaracy: 0.6285714285714286
Best parameters:
{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}


Der Extra-Trees-Klassifikator erreicht mit einer Accuracy von ca. 0,63 praktisch das gleiche Niveau wie der Random Forest (und etwas besser als der einzelne Entscheidungsbaum). Das passt zur Idee von Extra Trees: Durch stärkere Zufälligkeit bei den Splits sinkt die Varianz des Modells, ohne dass die Genauigkeit leidet. Trees liefert robuste, wettbewerbsfähige Ergebnisse bei geringem Tuning-Aufwand. Für die weitere Bewertung sollte man neben der Accuracy auch Konfusionsmatrix, Precision/Recall/F1 (insbesondere für die „bad“-Klasse) und ROC-AUC betrachten.

In [41]:
xgb = XGBClassifier(
    random_state=42,
    scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),    
    eval_metric="logloss",       
    n_jobs=-1,                 
    tree_method="hist"      
)

xgb_param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.2],
    "subsample": [0.7, 1.0],
    "colsample_bytree": [0.7, 1.0]
}

best_xgb, acc_xgb, params_xgb = train_model(
    xgb, xgb_param_grid, X_train, y_train, X_test, y_test
)

print(f"XGB Accuracy: {acc_xgb}")
print("Best parameters:", params_xgb)

XGB Accuracy: 0.6857142857142857
Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.7}


Da wir in der Praxis ein Modell haben wollen, das möglichst verlässliche Vorhersagen trifft, entscheiden ich mich für XGBoost. Für unsere Streamlit-Anwendung bedeutet das: Wir werden XGBoost als finales Modell exportieren und in die App einbauen. So können Nutzer dort später ihre Daten eingeben und eine Vorhersage über das Kreditrisiko bekommen – basierend auf dem leistungsstärksten Modell unserer Analyse.


In [42]:
# Speichern
joblib.dump(best_et, "extra_trees_credit_model.pkl")

['extra_trees_credit_model.pkl']